In [3]:
# import pandas as pd
import numpy as np
import pandas as pd

### Wandb Quickstart

In [4]:
import wandb
import random

# start a new wandb run to track this script
wandb.init(
    # set the wandb project where this run will be logged
    project="my-awesome-project",
    
    # track hyperparameters and run metadata
    config={
    "learning_rate": 0.02,
    "architecture": "CNN",
    "dataset": "CIFAR-100",
    "epochs": 10,
    }
)

# simulate training
epochs = 10
offset = random.random() / 5
for epoch in range(2, epochs):
    acc = 1 - 2 ** -epoch - random.random() / epoch - offset
    loss = 2 ** -epoch + random.random() / epoch + offset
    
    # log metrics to wandb
    wandb.log({"acc": acc, "loss": loss})
    
# [optional] finish the wandb run, necessary in notebooks
wandb.finish()

wandb: Currently logged in as: pashikantipranith7867. Use `wandb login --relogin` to force relogin


acc,▁▅▅█▇▇█▇
loss,█▃▂▃▂▂▂▁
acc,0.79281
loss,0.13783


### Explore

In [29]:
data = pd.read_csv("/home/pranith/Desktop/Development/Player_Performance_Prediction/CHECK/all_matches_v1.csv")

In [30]:
data.shape
data.sort_values(by='start_date', inplace=True)

In [31]:
data.head()

,batting_team,bowling_team,match_id,start_date,striker,Batsman_runs,Balls,Fours,Sixes,Fifties,Hundreds,Duck,dreamll_score,strike_rate,bowler,type
12226,Royal Challengers Bangalore,Kolkata Knight Riders,335982,2008-04-18,R Dravid,2,3,0.0,0.0,0,0,0,6.0,66.666667,MA Khote,caught
12222,Royal Challengers Bangalore,Kolkata Knight Riders,335982,2008-04-18,CL White,6,10,0.0,0.0,0,0,0,10.0,60.000000,JR Hazlewood,caught
12223,Royal Challengers Bangalore,Kolkata Knight Riders,335982,2008-04-18,JH Kallis,8,7,0.0,1.0,0,0,0,14.0,114.285714,Mohammed Shami,caught
12224,Royal Challengers Bangalore,Kolkata Knight Riders,335982,2008-04-18,MV Boucher,7,11,1.0,0.0,0,0,0,12.0,63.636364,YS Chahal,caught
12225,Royal Challengers Bangalore,Kolkata Knight Riders,335982,2008-04-18,P Kumar,18,17,1.0,2.0,0,0,0,27.0,105.882353,JR Hopes,lbw


In [32]:
data.tail()

,batting_team,bowling_team,match_id,start_date,striker,Batsman_runs,Balls,Fours,Sixes,Fifties,Hundreds,Duck,dreamll_score,strike_rate,bowler,type
10271,Rajasthan Royals,Gujarat Titans,1312200,2022-05-29,R Parag,15,15,1.0,0.0,0,0,0,20.0,100.000000,A Nehra,caught
10272,Rajasthan Royals,Gujarat Titans,1312200,2022-05-29,SO Hetmyer,11,12,2.0,0.0,0,0,0,17.0,91.666667,JJ Bumrah,caught
10275,Rajasthan Royals,Gujarat Titans,1312200,2022-05-29,YBK Jaiswal,22,16,1.0,2.0,0,0,0,31.0,137.500000,A Nehra,run out
10273,Rajasthan Royals,Gujarat Titans,1312200,2022-05-29,SV Samson,14,11,2.0,0.0,0,0,0,20.0,127.272727,Imran Tahir,caught
10270,Rajasthan Royals,Gujarat Titans,1312200,2022-05-29,R Ashwin,6,9,0.0,0.0,0,0,0,10.0,66.666667,HV Patel,caught


In [33]:

# filter rows for training set (2018-2020)
start_date_mask_train = (data['start_date'] >= '2018-01-01') & (data['start_date'] <= '2020-12-31')
data_train = data.loc[start_date_mask_train]

# filter rows for validation set (2021)
start_date_mask_val = (data['start_date'] >= '2021-01-01') & (data['start_date'] <= '2021-12-31')
data_val = data.loc[start_date_mask_val]

# filter rows for test set (2022)
start_date_mask_test = (data['start_date'] >= '2022-01-01') & (data['start_date'] <= '2022-12-31')
data_test = data.loc[start_date_mask_test]


In [34]:
print("A glimpse of training segment...")
# print(data_train)
data_train

A glimpse of training segment...


,batting_team,bowling_team,match_id,start_date,striker,Batsman_runs,Balls,Fours,Sixes,Fifties,Hundreds,Duck,dreamll_score,strike_rate,bowler,type
769,Chennai Super Kings,Mumbai Indians,1136561,2018-04-07,Imran Tahir,2,2,0.0,0.0,0,0,0,6.0,100.000000,A Nehra,caught and bowled
768,Chennai Super Kings,Mumbai Indians,1136561,2018-04-07,Harbhajan Singh,8,6,1.0,0.0,0,0,0,13.0,133.333333,Y Venugopal Rao,caught
767,Chennai Super Kings,Mumbai Indians,1136561,2018-04-07,DL Chahar,0,1,0.0,0.0,0,0,1,2.0,0.000000,DW Steyn,bowled
765,Chennai Super Kings,Mumbai Indians,1136561,2018-04-07,AT Rayudu,22,19,4.0,0.0,0,0,0,30.0,115.789474,Sohail Tanvir,caught
7614,Mumbai Indians,Chennai Super Kings,1136561,2018-04-07,E Lewis,0,2,0.0,0.0,0,0,1,2.0,0.000000,Kuldeep Yadav,stumped
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7811,Mumbai Indians,Delhi Capitals,1237181,2020-11-10,Ishan Kishan,33,19,3.0,1.0,0,0,0,42.0,173.684211,R Sharma,stumped
7810,Mumbai Indians,Delhi Capitals,1237181,2020-11-10,HH Pandya,3,5,0.0,0.0,0,0,0,7.0,60.000000,KA Pollard,caught
7816,Mumbai Indians,Delhi Capitals,1237181,2020-11-10,SA Yadav,19,20,1.0,1.0,0,0,0,26.0,95.000000,PP Ojha,run out
2273,Delhi Capitals,Mumbai Indians,1237181,2020-11-10,MP Stoinis,0,1,0.0,0.0,0,0,1,2.0,0.000000,P Awana,caught


In [36]:
data_train.to_csv("training_Data.csv")

In [35]:
print("A glimpse of validation segment...")
# print(data_train)
data_val

A glimpse of validation segment...


,batting_team,bowling_team,match_id,start_date,striker,Batsman_runs,Balls,Fours,Sixes,Fifties,Hundreds,Duck,dreamll_score,strike_rate,bowler,type
9004,Mumbai Indians,Royal Challengers Bangalore,1254058,2021-04-09,Ishan Kishan,28,20,2.0,1.0,0,0,0,36.0,140.000000,SR Watson,caught
9003,Mumbai Indians,Royal Challengers Bangalore,1254058,2021-04-09,HH Pandya,13,10,2.0,0.0,0,0,0,19.0,130.000000,Imran Tahir,caught
9005,Mumbai Indians,Royal Challengers Bangalore,1254058,2021-04-09,JJ Bumrah,1,2,0.0,0.0,0,0,0,5.0,50.000000,A Nehra,run out
9006,Mumbai Indians,Royal Challengers Bangalore,1254058,2021-04-09,KA Pollard,7,9,1.0,0.0,0,0,0,12.0,77.777778,Swapnil Singh,caught and bowled
9002,Mumbai Indians,Royal Challengers Bangalore,1254058,2021-04-09,CA Lynn,49,36,4.0,3.0,0,0,0,63.0,136.111111,JJ Bumrah,caught
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5854,Kolkata Knight Riders,Chennai Super Kings,1254117,2021-10-15,Shubman Gill,51,46,6.0,0.0,1,0,0,69.0,110.869565,CV Varun,bowled
5853,Kolkata Knight Riders,Chennai Super Kings,1254117,2021-10-15,Shivam Mavi,20,14,1.0,2.0,0,0,0,29.0,142.857143,K Rabada,caught
5855,Kolkata Knight Riders,Chennai Super Kings,1254117,2021-10-15,VR Iyer,50,32,5.0,3.0,1,0,0,73.0,156.250000,YS Chahal,caught
5851,Kolkata Knight Riders,Chennai Super Kings,1254117,2021-10-15,SP Narine,2,3,0.0,0.0,0,0,0,6.0,66.666667,I Sharma,caught


In [37]:
data_val.to_csv("validation_data.csv")
data_test.to_csv("Testing_Data.csv")